<a href="https://colab.research.google.com/github/codingwithsubhransu/Machine-Learning-Course/blob/main/Spam_Mail_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [8]:
raw_mail_data = pd.read_csv('/content/drive/MyDrive/Datasets/mail_data.csv')

In [9]:
raw_mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
raw_mail_data.shape

(5572, 2)

In [11]:
raw_mail_data['Category'].value_counts()

,count
Category,
ham,4825
spam,747


In [12]:
raw_mail_data.isnull().sum()

,0
Category,0
Message,0


In [13]:
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)), '')

In [14]:
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
mail_data.shape

(5572, 2)

Label Encoding

In [17]:
mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

spam -> 0

ham -> 1

In [18]:
x = mail_data['Message']
y = mail_data['Category']

In [19]:
print(x)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [20]:
print(y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)

In [22]:
print(x.shape, x_train.shape, x_test.shape)

(5572,) (4457,) (1115,)


Feature Extraction

In [24]:
feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
x_train_feature = feature_extraction.fit_transform(x_train)
x_test_feature = feature_extraction.transform(x_test)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [25]:
print(x_train_feature)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 34775 stored elements and shape (4457, 7431)>
  Coords	Values
  (0, 2329)	0.38783870336935383
  (0, 3811)	0.34780165336891333
  (0, 2224)	0.413103377943378
  (0, 4456)	0.4168658090846482
  (0, 5413)	0.6198254967574347
  (1, 3811)	0.17419952275504033
  (1, 3046)	0.2503712792613518
  (1, 1991)	0.33036995955537024
  (1, 2956)	0.33036995955537024
  (1, 2758)	0.3226407885943799
  (1, 1839)	0.2784903590561455
  (1, 918)	0.22871581159877646
  (1, 2746)	0.3398297002864083
  (1, 2957)	0.3398297002864083
  (1, 3325)	0.31610586766078863
  (1, 3185)	0.29694482957694585
  (1, 4080)	0.18880584110891163
  (2, 6601)	0.6056811524587518
  (2, 2404)	0.45287711070606745
  (2, 3156)	0.4107239318312698
  (2, 407)	0.509272536051008
  (3, 7414)	0.8100020912469564
  (3, 2870)	0.5864269879324768
  (4, 2870)	0.41872147309323743
  (4, 487)	0.2899118421746198
  :	:
  (4454, 2855)	0.47210665083641806
  (4454, 2246)	0.47210665083641806
  (4455, 4456)	0.24

In [26]:
model = LogisticRegression()

In [27]:
model.fit(x_train_feature, y_train)

LogisticRegression()

In [28]:
# Evaluating the model
prediction_on_training = model.predict(x_train_feature)
accuracy_training = accuracy_score(y_train, prediction_on_training)

In [29]:
print(accuracy_training)

0.9676912721561588


In [30]:
prediction_on_testing = model.predict(x_test_feature)
accuracy_testing = accuracy_score(y_test, prediction_on_testing)

In [31]:
print(accuracy_testing)

0.9668161434977578


Building a prediction system

In [34]:
input_mail = ["Hi everyone, Just a friendly reminder about the family reunion this Saturday at the park. Please let me know if you can make it by Wednesday so we can get a headcount. Looking forward to seeing you all! Love, Aunt"]
input_mail_feature = feature_extraction.transform(input_mail)
prediction = model.predict(input_mail_feature)
print(prediction)

if prediction[0] == 1:
  print('Ham mail')
else:
  print('Spam mail')

[1]
Ham mail


In [36]:
import pickle
from sklearn.linear_model import LogisticRegression

filename = '/content/drive/MyDrive/Spam Mail Prediction Model/spam_mail_prediction_model.pkl'
pickle.dump(model, open(filename, 'wb'))